# Text2vec-base-multilingual

In [1]:
#!pip install ktrain

In [2]:
#!pip install sentence_transformers

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import ktrain
from ktrain import text

#from sentence_transformers import SentenceTransformer

data_set = pd.read_csv(r"C:\Users\volpe\Documents\MARCO\0.master\0.Corsi\Machine_learning\Carman_google_drive\TextMining\data_0905_semicolon.csv",sep=";")

c:\Users\volpe\anaconda3\envs\kt\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


https://huggingface.co/shibing624/text2vec-base-multilingual

In [4]:
data_set.head()

,Unnamed: 0,Ticker,Titles,Date,Var_prc,Target
0,0,MMM,<title>Here is What to Know Beyond Why 3M Comp...,0,0.008478,0.0
1,69,MMM,<title>What's Next For 3M Stock After A 15% Fa...,0,0.008478,0.0
2,68,MMM,<title>MMM Stock Quote Price and Forecast - CN...,0,0.008478,0.0
3,67,MMM,<title>3M Announces Upcoming Investor Event - ...,0,0.008478,0.0
4,66,MMM,<title>Open Door: Claire Pentecost + Beth Jaco...,0,0.008478,0.0


In [5]:
# we want a input dataframe where there is only a row for each ticker, 10 columns with the 10 titles, and the target
number_articles = 10
col =["Ticker"] + [ "Article " + str(i+1)  for i in range(number_articles)] + ["Target"]
input_df = pd.DataFrame(columns=col)

for ticker in data_set["Ticker"].unique():
    #extract the rows corresponfing to the ticker
    ticker_df = data_set[ data_set["Ticker"] ==  ticker ]
    #create the new row to append to the dataset
    articles_list = ticker_df["Titles"].to_list()
    ticker_target = ticker_df["Target"].iloc[0]
    new_row = [ticker] + articles_list + [ticker_target] 
    #append the new row to the dataset
    input_df = input_df.append(pd.Series(new_row, index=col),ignore_index=True)
input_df = input_df.dropna()
print(input_df.shape)
input_df.iloc[60:63,:]

(498, 12)


,Ticker,Article 1,Article 2,Article 3,Article 4,Article 5,Article 6,Article 7,Article 8,Article 9,Article 10,Target
60,BAX,<title>Biden is tweaking an asylum rule to spe...,<title>Baxter's (BAX) Latest FDA Approval to B...,<title>Mayor Sarno Joined with Bax and Nagle t...,"<title>Based On Its ROE, Is Baxter Internation...",<title>(01/02/24) Top Picks 2024: Baxter Inter...,"<title>Baxter (BAX) Beats on Q3 Earnings, Dive...",<title>Is Baxter International (BAX) Too Good ...,<title>Baxter International Inc.'s (NYSE:BAX) ...,<title>Author Correction: Bax Inhibitor-1 regu...,<title>Estimating The Fair Value Of Baxter Int...,-1.0
61,BDX,"<title>When the Price of (BDX) Talks, People L...","<title>Becton, Dickinson and Company (NYSE:BDX...",<title>Beyond the Balance Sheet: What SWOT Rev...,<title>Becton Dickinson (BDX) Surpasses Q3 Ear...,<title>Breaking: BDx Indonesia to acquire Indo...,<title>BD Increases Access to Cutting-Edge Ima...,<title>Investors Still Waiting For A Pull Back...,<title>BDx acquires data center freehold in Si...,<title>BDX Droids Coming to Star Wars Galaxy's...,<title>Why Becton Dickinson (BDX) is a Top Val...,-1.0
63,BBY,<title>Best Buy (NYSE:BBY) PT Lowered to $67.0...,<title>Best Buy (BBY) Up 11.1% Since Last Earn...,<title>Best Buy Reports Q4 FY24 Results - Best...,<title>Decoding Best Buy Co Inc (BBY): A Strat...,<title>Should You Think About Buying Best Buy ...,<title>This Insider Has Just Sold Shares In Be...,<title>Best Buy Geek Squad employees report ma...,<title>Best Buy Recalls Insignia® Air Fryers a...,"<title>Down 15% This Year, Where Is Best Buy S...",<title>Dividend Investors: Don't Be Too Quick ...,0.0


In [6]:
# Cleaning the columns and creating the input for the multilingual bert
for i in range(1, 11):
    column_name = f'Article {i}'
    input_df[column_name] = input_df[column_name].str.replace('<title>', '').str.replace('</title>', '')

input_df["Concatenated Titles"] = input_df.iloc[:, 1:11].apply(lambda row: ' '.join(row), axis=1)
input_df.dropna(axis = 0)
# Print the resulting DataFrame
input_df.iloc[50:53,:]

,Ticker,Article 1,Article 2,Article 3,Article 4,Article 5,Article 6,Article 7,Article 8,Article 9,Article 10,Target,Concatenated Titles
50,ADP,Ep. 539 - Analyzing the New Post-Draft Startup...,ADP Increases Cash Dividend; Marks 49th Consec...,Why Is Software Megacap ADP's Stock Trading Hi...,The Biggest ADP Fallers: How long Gerrit Cole ...,ADP Ventures | Creating a better world of work...,"Private payrolls rose by 140,000 in February, ...",ADP Reimagines the Human Capital Management Cl...,ADP: Employers Added 103000 Workers in Novembe...,ADP Declares Regular Quarterly Dividend - PR N...,ADP vs Paychex: Which Payroll Software is Bett...,0.0,Ep. 539 - Analyzing the New Post-Draft Startup...
51,AZO,(AZO) Investment Report - Stock Traders Daily,Light-tuned selective photosynthesis of azo- a...,Optimal of black anti-static AZO/Al2O3-SiO2-Vx...,"New azo-azomethine derivatives: Synthesis, cha...",NLO azo compounds with sulfonamide groups: A t...,Self-organized patterning on azo molecular gla...,"Synthesis, spectroscopic characterization and ...",Protocol for photoelectrocatalytic synthesis o...,Easily accessible and tunable porous organic p...,"Reversal in solvatochromism, photochromism and...",0.0,(AZO) Investment Report - Stock Traders Daily ...
52,AVB,Decoding AvalonBay Communities Inc (AVB): A St...,'Massive arguments' - AVB reveals Tottenham re...,Van der Vaart calls AVB 'an idiot' and explain...,10 Ways to Use AVB ('Already Vaped Bud') Marij...,Ruben Amorim to Liverpool: Will he become a Mo...,AVIATION LOGISTICS SUPPORT IN THE INDO-PACIFIC...,6 Easy Ways for Getting High Again with Your A...,‘This is really uncomfortable’ – John Terry an...,InfoComm 2023: DiGiCo Introduces Milan-Certifi...,Analysts See Upside On These 3 REITs - AvalonB...,0.0,Decoding AvalonBay Communities Inc (AVB): A St...


In [7]:
from sklearn.model_selection import train_test_split

X = input_df.filter(["Concatenated Titles"],axis=1)
Y = input_df.filter(["Target"],axis=1)
#print(Y)

x_train, x_test, y_train, y_test = train_test_split( X, Y, test_size=0.20, random_state=42)

### Here starts the model training

In [8]:
model_name = 'mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis'
transformer = text.Transformer(model_name, maxlen=512, class_names= np.array([0,1,2]))
y_train = y_train + 1          # classes need to start from
y_test = y_test + 1

In [ ]:
processed_train = transformer.preprocess_train(list(x_train["Concatenated Titles"]), list(y_train["Target"]))
processed_test = transformer.preprocess_test(list(x_test["Concatenated Titles"]), list(y_test["Target"]))

In [ ]:
model = transformer.get_classifier()
batch_size = 5                        #small training dataset
learner = ktrain.get_learner(model, train_data=processed_train, val_data=processed_test, batch_size=batch_size)

In [ ]:
print(processed_train.y)

In [ ]:
learner.lr_find(show_plot=True,  max_epochs=1)

In [ ]:
learner.fit_onecycle(0.5e-1, 2)

In [ ]:
learner.validate(class_names=transformer.get_classes())

In [ ]:
'''
# let's see some predictions
'''
predictor = ktrain.get_predictor(learner.model, preproc=transformer)
import random as rand
i = rand.randint(0,100)

print(input_df["Ticker"].iloc[i],"\n",\
      "Prediction: \n",\
      predictor.predict(x_test["Concatenated Titles"].iloc[i]),\
      "\nTarget: \n",\
      y_train["Target"].iloc[i])

### Let's try Doc2vect + NN

In [ ]:
from sklearn.model_selection import train_test_split
m = SentenceTransformer("shibing624/text2vec-base-multilingual")

X_df = input_df.filter(["Titles"],axis=1)
Y_df = input_df.filter(["Target"],axis=1)
#print(Y_df)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/88.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/496 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
c = 384 # dim embedding
r = 500 # dim dataset

X = np.array([ [0]*c   ] *  r)
for idx,row in input_df.iterrows():
  sentence_embeddings = m.encode(row["Titles"])
  X[idx,:] = sentence_embeddings


In [ ]:
t_size = 0.2

x_train, x_test, y_train, y_test = train_test_split( X, Y_df, test_size=t_size, random_state=42)

In [ ]:
y_train = y_train + 1           # VUOLE CLASSI NUMERATE DA 0
y_test = y_test + 1
#processed_train = transformer.preprocess_train(x_train, list(y_train["Target"]))
#processed_test = transformer.preprocess_test(x_train, list(y_test["Target"]))

In [ ]:
len(X[0,:])
print(len(y_train),"---",y_train["Target"].iloc[2])
print(type(y_train))
for i in range(0,len(y_train)):
  pass
  #print(y_train["Target"].iloc[i])

400 --- 1
<class 'pandas.core.frame.DataFrame'>


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam, RMSprop, SGD, Adagrad

In [ ]:
#to train the NN we need to convert integer output to binary array
num_classes = 3
y_train_dummy = np.zeros((len(y_train), num_classes))


for i in range(0,len(y_train)):
    class_idx = y_train.iloc[i]["Target"]  # Assuming "Target" is the column name
    y_train_dummy[i, class_idx] = 1

y_test_dummy = np.zeros((len(y_test), num_classes))
for i in range(0,len(y_test)):
  y_test_dummy[i,y_test["Target"].iloc[i]] = 1


In [ ]:
model=Sequential()
model.add(Dense(units=384, activation='linear', input_dim=384))
model.add(Dense(units=200, activation='relu'))
model.add(Dense(units=3, activation='sigmoid'))    #output layer
model.compile(loss='mse',
              optimizer=Adam(lr=0.001,beta_1=0.9,beta_2=0.999))

model.fit(x_train,y_train,epochs=2000,batch_size=6)  #10, 5


pred_train=model.predict(x_train)
pred_test=model.predict(x_test)

In [ ]:
pred_test_2 = np.zeros(pred_test.shape)
for i in range(len(pred_test)):
  pred_test_2[i, pred_test[i,:].argmax()  ] = 1

pred_train_2 = np.zeros(pred_train.shape)
for i in range(len(pred_train)):
  pred_train_2[i, pred_train[i,:].argmax()  ] = 1

In [ ]:
print(pred_train.shape,"--",x_train.shape)
print(pred_test.shape,"--",x_test.shape)

print(pred_train[23,:],"--",y_train_dummy[0,:])
print(  )
for i in range(160,200):
  print(pred_train_2[i,:],"--",y_train_dummy[i,:])

(400, 3) -- (400, 384)
(100, 3) -- (100, 384)
[1. 1. 1.] -- [1. 0. 0.]

[1. 0. 0.] -- [0. 1. 0.]
[1. 0. 0.] -- [0. 1. 0.]
[1. 0. 0.] -- [0. 1. 0.]
[1. 0. 0.] -- [0. 1. 0.]
[1. 0. 0.] -- [0. 0. 1.]
[1. 0. 0.] -- [0. 0. 1.]
[1. 0. 0.] -- [0. 0. 1.]
[1. 0. 0.] -- [0. 1. 0.]
[1. 0. 0.] -- [0. 1. 0.]
[1. 0. 0.] -- [0. 1. 0.]
[1. 0. 0.] -- [0. 0. 1.]
[1. 0. 0.] -- [0. 1. 0.]
[1. 0. 0.] -- [0. 1. 0.]
[1. 0. 0.] -- [0. 1. 0.]
[1. 0. 0.] -- [0. 1. 0.]
[1. 0. 0.] -- [0. 0. 1.]
[1. 0. 0.] -- [0. 1. 0.]
[1. 0. 0.] -- [0. 1. 0.]
[1. 0. 0.] -- [0. 1. 0.]
[1. 0. 0.] -- [0. 1. 0.]
[1. 0. 0.] -- [0. 1. 0.]
[1. 0. 0.] -- [0. 0. 1.]
[1. 0. 0.] -- [0. 1. 0.]
[1. 0. 0.] -- [0. 1. 0.]
[1. 0. 0.] -- [0. 1. 0.]
[1. 0. 0.] -- [0. 1. 0.]
[1. 0. 0.] -- [0. 0. 1.]
[1. 0. 0.] -- [0. 0. 1.]
[1. 0. 0.] -- [0. 0. 1.]
[1. 0. 0.] -- [0. 0. 1.]
[1. 0. 0.] -- [0. 1. 0.]
[1. 0. 0.] -- [0. 1. 0.]
[1. 0. 0.] -- [1. 0. 0.]
[1. 0. 0.] -- [0. 1. 0.]
[1. 0. 0.] -- [0. 1. 0.]
[1. 0. 0.] -- [0. 1. 0.]
[1. 0. 0.] -- [0. 0. 1.]
[1.

In [ ]:
MSE_train = np.sum(  (pred_train-y_train_dummy)**2)   /pred_train.shape[0]
MSE_test = np.sum(  (pred_test-y_test_dummy)**2)   /pred_test.shape[0]

print("MSE_train: ",MSE_train)
print("MSE_test: ",MSE_test)

MSE_train = np.sum(  (pred_train_2-y_train_dummy)**2)   /pred_train.shape[0]
MSE_test = np.sum(  (pred_test_2-y_test_dummy)**2)   /pred_test.shape[0]

print("MSE_train: ",MSE_train)
print("MSE_test: ",MSE_test)

MSE_train:  2.0
MSE_test:  2.0


In [ ]:
miss_train = np.sum(  abs(pred_train_2-y_train_dummy)/2)   /pred_train.shape[0]
miss_test = np.sum(  abs(pred_test_2-y_test_dummy)/2)   /pred_test.shape[0]

print("Missclasification_train: ",miss_train)
print("Missclassification_test: ",miss_test)

Missclasification_train:  0.9325
Missclassification_test:  0.93


In [ ]:
'''
model=Sequential()
model.add(Dense(units=384, activation='relu', input_dim=384))
model.add(Dense(units=200, activation='tanh'))
model.add(Dense(units=200, activation='sigmoid'))  ##
model.add(Dense(units=200, activation='linear'))  ##
model.add(Dense(units=3, activation='sigmoid'))    #output layer
model.compile(loss='mse',
              optimizer=Adam(lr=0.0001,beta_1=0.9,beta_2=0.999))

model.fit(x_train,y_train,epochs=200,batch_size=10)  #10, 5


pred_train=model.predict(x_train)
pred_test=model.predict(x_test)


MSE_train:  1.9999998185038634
MSE_test:  1.9999998080730506

MSE_train:  0.875
MSE_test:  0.7

Missclasification_train:  0.4375
Missclassification_test:  0.35

'''

"\nmodel=Sequential()\nmodel.add(Dense(units=384, activation='relu', input_dim=384))\nmodel.add(Dense(units=200, activation='tanh'))\nmodel.add(Dense(units=200, activation='sigmoid'))  ##\nmodel.add(Dense(units=200, activation='linear'))  ##\nmodel.add(Dense(units=3, activation='sigmoid'))    #output layer\nmodel.compile(loss='mse',\n              optimizer=Adam(lr=0.0001,beta_1=0.9,beta_2=0.999))\n\nmodel.fit(x_train,y_train,epochs=200,batch_size=10)  #10, 5\n\n\npred_train=model.predict(x_train)\npred_test=model.predict(x_test)\n\n\nMSE_train:  1.9999998185038634\nMSE_test:  1.9999998080730506\n\nMSE_train:  0.875\nMSE_test:  0.7\n\nMissclasification_train:  0.4375\nMissclassification_test:  0.35\n\n"